## Gravitino Trino Example

In this example, we will use `Jupyter` and the `Trino Python Client` to experience `Gravitino`.

In [1]:
# install trino python client and pandas
%pip install trino==0.335.0 pandas

  Using cached trino-0.336.0-py3-none-any.whl.metadata (20 kB)
  Using cached pandas-2.3.1-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (91 kB)
  Using cached lz4-4.4.4-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (3.8 kB)
  Using cached orjson-3.11.2-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (1.2 kB)
  Using cached pytz-2025.2-py2.py3-none-any.whl.metadata (22 kB)
  Using cached requests-2.32.5-py3-none-any.whl.metadata (4.9 kB)
  Using cached tzlocal-5.3.1-py3-none-any.whl.metadata (7.6 kB)
  Using cached zstandard-0.24.0-cp312-cp312-manylinux2014_x86_64.manylinux_2_17_x86_64.whl.metadata (3.1 kB)
Using cached trino-0.336.0-py3-none-any.whl (57 kB)
Using cached pandas-2.3.1-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (12.0 MB)
Using cached orjson-3.11.2-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (121 kB)
Using cached pytz-2025.2-py2.py3-none-any.whl (509 kB)
Using cached requests-2.

In [4]:
from trino.dbapi import connect
import os

# Create a Trino connector client
conn = connect(
    host="localhost",
    port=8080,
    user="admin",
    catalog="catalog_hive",
    schema="http",
)

trino_client = conn.cursor()

## Prepare

Creates a schema named `catalog_hive.company` in Hive, with its location set to`hdfs://hive:9000/user/hive/warehouse/company.db` on HDFS.

In [ ]:
import os

trino_client.execute("""
CREATE SCHEMA catalog_hive.company
  WITH (location = 'hdfs://hive:9000/user/hive/warehouse/company.db')
""").fetchall()

Displays the SQL command that was used to create the schema `catalog_hive.company`.

In [ ]:
trino_client.execute("""
SHOW CREATE SCHEMA catalog_hive.company
""").fetchall()

Create `employees` table

In [ ]:
# Create Table
trino_client.execute(
"""
CREATE TABLE catalog_hive.company.employees
(
  name varchar,
  salary decimal(10,2)
)
WITH (
  format = 'TEXTFILE'
)
"""
).fetchall()

In [ ]:
# Insert data
print(trino_client.execute("INSERT INTO catalog_hive.company.employees (name, salary) VALUES ('Sam Evans', 55000)").fetchall())

## Simple queries

Some simple query testing.

In [ ]:
import pandas as pd

# Show employees table contents
df = pd.DataFrame(trino_client.execute("SELECT * FROM catalog_hive.company.employees").fetchall(), columns=['Name', 'Salary'])

# Display the DataFrame
df

In [ ]:
# Execute the queries and convert the results directly to DataFrames
df_g = pd.DataFrame(trino_client.execute("SHOW SCHEMAS from catalog_hive").fetchall(), columns=['Schema'])
df_g

In [ ]:
h = trino_client.execute("DESCRIBE catalog_hive.company.employees").fetchall()
h

In [ ]:
df_i = pd.DataFrame(trino_client.execute("SHOW TABLES from catalog_hive.company").fetchall(), columns=['Tables'])
df_i

## Cross-catalog queries

In a company, there may be different departments using different data stacks. In this example, the HR department uses Apache Hive to store its data and the sales department uses PostgreSQL. You can run some interesting queries by joining the two departments' data together with Gravitino.

To know which employee has the largest sales amount:

In [ ]:
# Cross-catalog queries
cross_catalog = trino_client.execute("""
SELECT given_name, family_name, job_title, sum(total_amount) AS total_sales
FROM catalog_hive.sales.sales as s,
  catalog_postgres.hr.employees AS e
where s.employee_id = e.employee_id
GROUP BY given_name, family_name, job_title
ORDER BY total_sales DESC
LIMIT 1
""").fetchall()

# Convert the result to a DataFrame
df_j = pd.DataFrame(cross_catalog, columns=['Given Name', 'Family Name', 'Job Title', 'Total Sales'])

df_j

To know the top customers who bought the most by state:

In [ ]:
# Execute the query
k = trino_client.execute("""
SELECT customer_name, location, SUM(total_amount) AS total_spent
FROM catalog_hive.sales.sales AS s,
  catalog_hive.sales.stores AS l,
  catalog_hive.sales.customers AS c
WHERE s.store_id = l.store_id AND s.customer_id = c.customer_id
GROUP BY location, customer_name
ORDER BY location, SUM(total_amount) DESC
""").fetchall()

# Convert the result to a DataFrame
df_k = pd.DataFrame(k, columns=['Customer Name', 'Location', 'Total Spent'])

# Display the DataFrame
df_k

To know the employee's average performance rating and total sales:

In [ ]:
# Execute the query
l = trino_client.execute("""
SELECT e.employee_id, given_name, family_name, AVG(rating) AS average_rating, SUM(total_amount) AS total_sales
FROM catalog_postgres.hr.employees AS e,
  catalog_postgres.hr.employee_performance AS p,
  catalog_hive.sales.sales AS s
WHERE e.employee_id = p.employee_id AND p.employee_id = s.employee_id
GROUP BY e.employee_id,  given_name, family_name
""").fetchall()

# Convert the result to a DataFrame
df_l = pd.DataFrame(l, columns=['Employee ID', 'Given Name', 'Family Name', 'Average Rating', 'Total Sales'])

# Display the DataFrame
df_l